# AI(2A) - Human Robot Interaction Project
# 👩‍⚕️  Nurse Robot

## Introduction
This script implements a Spoke Dialog System (SDS) based on Spoken Natural Language Dialog in the context of an automated home nurse assistant, to be used by people with limited capabilities. It simulates actions such as reaching things, provide meals and inspect patient status. The system acquires a spoken input from the user through Google Speech Recognition API. Then, recognizes the intent of the sentence using Rasa NLU and retrieves and action result that is transmitted to the user through VocalWare Text-To-Speech API. The order of the sentences, and the recognition of the utterances and dependencies is processed by the libraries spaCy and Keras, building a model that returns the adequate actions given a series of inputs. 

![title](images/NurseRobotMindmap.png)

In [1]:
#imports for project

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import logging
import rasa_core
from rasa_core.agent import Agent
from rasa_core.policies.keras_policy import KerasPolicy
from rasa_core.policies.memoization import MemoizationPolicy
from rasa_core.interpreter import RasaNLUInterpreter
from rasa_nlu.model import Metadata, Interpreter
from rasa_core.utils import EndpointConfig
from rasa_core.run import serve_application
from rasa_nlu.model import Trainer
from rasa_nlu.training_data import load_data
from rasa_core import config
from IPython.display import IFrame
import pandas as pd
import ruamel
import urllib.request
from urllib.request import urlopen
from playsound import playsound
import uuid
import speech_recognition as sr
import os
import spacy
from spacy import displacy
import warnings
warnings.simplefilter('ignore', ruamel.yaml.error.UnsafeLoaderWarning)
warnings.filterwarnings("ignore",category=DeprecationWarning)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' #To ignore Tensorflow AVX AVX2 bonary warning

nlp = spacy.load('en')


Some of the data used in this project is not shown in this notebook, but in external files. The content of each relevant file is described hereunder.

* nurse_domain.yml: Defines the actions that can be taken by the agent, the intents that are expected, the entities to be saved if received, and some answers to the basic user utterances.
* stories.md: Contains the examples of dialogs and the specification of the entities that could be found on the phrases.
* config_spacy.json: Saves the pipeline to be used by the NLP library, in this case sklearn with spaCy 


In [2]:
# Declare paths
domain_file = './nurse_domain.yml'
model_path = './models/dialogue'
interpreter_path ='./models/nursebot/interpreter'
training_data_file = './data/stories.md'
conf_file = './config_spacy.json'

In [3]:
# Build the interpreter of the elements of dialog for the agent
#!python -m rasa_nlu.train -c config_spacy.json --data data/data.md -o models --fixed_model_name interpreter --project nursebot --verbose

In [31]:
action_endpoint = EndpointConfig(url="http://localhost:5055/webhook")
interpreter = RasaNLUInterpreter(interpreter_path)

agent = Agent(domain_file, 
              policies = [MemoizationPolicy(), KerasPolicy(max_history=3, epochs=200, batch_size=50)],
              interpreter=interpreter,
              action_endpoint=action_endpoint)
data = agent.load_data(training_data_file)
agent.train(data)

Processed Story Blocks: 100%|████████████████████████████████████████████████| 30/30 [00:00<00:00, 155.83it/s, # trackers=18]
Processed actions: 2306it [00:03, 612.17it/s, # examples=2242]


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, 3, 36)             0         
_________________________________________________________________
lstm (LSTM)                  (None, 32)                8832      
_________________________________________________________________
dense (Dense)                (None, 16)                528       
_________________________________________________________________
activation (Activation)      (None, 16)                0         
Total params: 9,360
Trainable params: 9,360
Non-trainable params: 0
_________________________________________________________________
Epoch 1/200
1228/1228 [==============================] - ETA: 15s - loss: 2.7788 - acc: 0.02 - ETA: 0s - loss: 2.6763 - acc: 0.2988 - 1s 616us/step - loss: 2.6421 - acc: 0.3428
Epoch 2/200
1228/1228 [==============================] - ETA: 0s - loss: 2.5647 - acc: 0.360 - 

1228/1228 [==============================] - ETA: 0s - loss: 0.1392 - acc: 0.980 - ETA: 0s - loss: 0.2242 - acc: 0.936 - 0s 69us/step - loss: 0.2260 - acc: 0.9414
Epoch 43/200
1228/1228 [==============================] - ETA: 0s - loss: 0.2068 - acc: 0.940 - ETA: 0s - loss: 0.2037 - acc: 0.945 - 0s 72us/step - loss: 0.2035 - acc: 0.9471
Epoch 44/200
1228/1228 [==============================] - ETA: 0s - loss: 0.2925 - acc: 0.900 - ETA: 0s - loss: 0.1986 - acc: 0.943 - 0s 64us/step - loss: 0.2140 - acc: 0.9422
Epoch 45/200
1228/1228 [==============================] - ETA: 0s - loss: 0.1160 - acc: 0.960 - ETA: 0s - loss: 0.2180 - acc: 0.943 - 0s 65us/step - loss: 0.2132 - acc: 0.9454
Epoch 46/200
1228/1228 [==============================] - ETA: 0s - loss: 0.1139 - acc: 0.980 - ETA: 0s - loss: 0.1957 - acc: 0.952 - 0s 59us/step - loss: 0.1970 - acc: 0.9511
Epoch 47/200
1228/1228 [==============================] - ETA: 0s - loss: 0.1351 - acc: 0.920 - ETA: 0s - loss: 0.1439 - acc: 0.964 -

1228/1228 [==============================] - ETA: 0s - loss: 0.0627 - acc: 1.000 - ETA: 0s - loss: 0.0896 - acc: 0.976 - 0s 66us/step - loss: 0.0826 - acc: 0.9764
Epoch 89/200
1228/1228 [==============================] - ETA: 0s - loss: 0.1000 - acc: 0.940 - ETA: 0s - loss: 0.0933 - acc: 0.972 - 0s 56us/step - loss: 0.0958 - acc: 0.9691
Epoch 90/200
1228/1228 [==============================] - ETA: 0s - loss: 0.1368 - acc: 0.940 - ETA: 0s - loss: 0.0867 - acc: 0.971 - 0s 67us/step - loss: 0.0809 - acc: 0.9723
Epoch 91/200
1228/1228 [==============================] - ETA: 0s - loss: 0.1238 - acc: 0.960 - ETA: 0s - loss: 0.1046 - acc: 0.962 - 0s 64us/step - loss: 0.0953 - acc: 0.9691
Epoch 92/200
1228/1228 [==============================] - ETA: 0s - loss: 0.0205 - acc: 1.000 - ETA: 0s - loss: 0.0697 - acc: 0.976 - 0s 61us/step - loss: 0.0747 - acc: 0.9780
Epoch 93/200
1228/1228 [==============================] - ETA: 0s - loss: 0.0955 - acc: 0.980 - ETA: 0s - loss: 0.0831 - acc: 0.981 -

1228/1228 [==============================] - ETA: 0s - loss: 0.0281 - acc: 1.000 - ETA: 0s - loss: 0.0536 - acc: 0.981 - 0s 57us/step - loss: 0.0518 - acc: 0.9837
Epoch 135/200
1228/1228 [==============================] - ETA: 0s - loss: 0.0679 - acc: 0.960 - ETA: 0s - loss: 0.0573 - acc: 0.985 - 0s 62us/step - loss: 0.0526 - acc: 0.9870
Epoch 136/200
1228/1228 [==============================] - ETA: 0s - loss: 0.0305 - acc: 1.000 - ETA: 0s - loss: 0.0518 - acc: 0.983 - 0s 58us/step - loss: 0.0514 - acc: 0.9829
Epoch 137/200
1228/1228 [==============================] - ETA: 0s - loss: 0.0275 - acc: 1.000 - ETA: 0s - loss: 0.0544 - acc: 0.982 - 0s 58us/step - loss: 0.0585 - acc: 0.9821
Epoch 138/200
1228/1228 [==============================] - ETA: 0s - loss: 0.0195 - acc: 1.000 - ETA: 0s - loss: 0.0477 - acc: 0.982 - 0s 61us/step - loss: 0.0507 - acc: 0.9805
Epoch 139/200
1228/1228 [==============================] - ETA: 0s - loss: 0.0190 - acc: 1.000 - ETA: 0s - loss: 0.0526 - acc: 0.

1228/1228 [==============================] - ETA: 0s - loss: 0.0394 - acc: 0.980 - ETA: 0s - loss: 0.0367 - acc: 0.987 - 0s 69us/step - loss: 0.0390 - acc: 0.9845
Epoch 181/200
1228/1228 [==============================] - ETA: 0s - loss: 0.0105 - acc: 1.000 - ETA: 0s - loss: 0.0349 - acc: 0.985 - 0s 65us/step - loss: 0.0346 - acc: 0.9862
Epoch 182/200
1228/1228 [==============================] - ETA: 0s - loss: 0.0112 - acc: 1.000 - ETA: 0s - loss: 0.0382 - acc: 0.986 - 0s 58us/step - loss: 0.0368 - acc: 0.9870
Epoch 183/200
1228/1228 [==============================] - ETA: 0s - loss: 0.0447 - acc: 0.980 - ETA: 0s - loss: 0.0501 - acc: 0.982 - 0s 58us/step - loss: 0.0481 - acc: 0.9813
Epoch 184/200
1228/1228 [==============================] - ETA: 0s - loss: 0.0047 - acc: 1.000 - ETA: 0s - loss: 0.0384 - acc: 0.986 - 0s 58us/step - loss: 0.0381 - acc: 0.9878
Epoch 185/200
1228/1228 [==============================] - ETA: 0s - loss: 0.0069 - acc: 1.000 - ETA: 0s - loss: 0.0316 - acc: 0.

In [33]:
# Save model to be used later
agent.persist(model_path)

In [34]:
# Load saved agent, in case no training is needed.
agent = Agent.load('./models/dialogue', interpreter=interpreter, action_endpoint=action_endpoint)

In [7]:
# Generate tree of interaction of utterances in possible dialogs
treeFile = "images/dialogtree.html"
agent.visualize("data/stories.md",output_file=treeFile, max_history=2)
IFrame(src=treeFile, width=1000, height=600)

Processed Story Blocks: 100%|█████████████████████████████████████████████████| 30/30 [00:00<00:00, 455.76it/s, # trackers=1]


In [18]:
interpreter.parse(u"Pass the remote") # Parsed phrase

{'intent': {'name': 'reach', 'confidence': 0.503009711233227},
 'entities': [{'start': 9,
   'end': 15,
   'value': 'remote',
   'entity': 'object',
   'confidence': 0.9015564182515755,
   'extractor': 'ner_crf'}],
 'intent_ranking': [{'name': 'reach', 'confidence': 0.503009711233227},
  {'name': 'move', 'confidence': 0.21193673058601706},
  {'name': 'call', 'confidence': 0.11101750540756687},
  {'name': 'status', 'confidence': 0.040966139842331675},
  {'name': 'attention', 'confidence': 0.02785851299136267},
  {'name': 'thanks', 'confidence': 0.027307996844929724},
  {'name': 'deny', 'confidence': 0.018137969234439215},
  {'name': 'joke', 'confidence': 0.017821324100716216},
  {'name': 'goodbye', 'confidence': 0.016981845815809575},
  {'name': 'greet', 'confidence': 0.015907384989873076}],
 'text': 'Pass the remote'}

In [19]:
# Interpret a sentence
def parsetxt(sent, deptree = False):
    pp = interpreter.parse(sent) # Parsed phrase
    # Print entities present in the sentence
    print(pd.DataFrame(pp['entities']))
    df = pd.DataFrame(pp['intent_ranking']).style.apply(lambda x: ['background: lightgreen' if x.name == 0 else '' for i in x],axis=1)
    if(deptree):
        doc = nlp(sent)
        displacy.render(doc, style='dep', jupyter=True, options={'distance':90})
    return df

In [20]:
# Interpret a sentence
sent1 = u"Nurse, can you pass me the remote and my book, please"
parsetxt(sent1)

   confidence  end  entity extractor  start   value
0    0.752938   33  object   ner_crf     27  remote
1    0.809981   45  object   ner_crf     41    book


,confidence,name
0,0.554473,reach
1,0.0728676,status
2,0.0651023,thanks
3,0.0630553,move
4,0.0535555,joke
5,0.0531397,call
6,0.0499545,attention
7,0.0272511,goodbye
8,0.0235214,deny
9,0.0218626,greet


In [21]:
# Render Dependency Tree
doc = nlp(sent1)
displacy.render(doc, style='dep', jupyter=True, options={'distance':90})

In [22]:
# Interprete a sentence
sent2 = "Nurse, help me to get to the bathroom please"
parsetxt(sent2)

   confidence  end entity extractor  start     value
0    0.900694   37  place   ner_crf     29  bathroom


,confidence,name
0,0.449045,move
1,0.185323,joke
2,0.108928,reach
3,0.0658736,call
4,0.0525246,status
5,0.0343769,thanks
6,0.0331593,attention
7,0.0237619,goodbye
8,0.020277,greet
9,0.0177115,deny


In [23]:
# Render Dependency Tree
doc = nlp(sent2)
displacy.render(doc, style='dep', jupyter=True, options={'distance':110})

In [24]:
parsetxt("Call my daughter")

   confidence  end  entity extractor  start     value
0     0.83458   16  callto   ner_crf      8  daughter


,confidence,name
0,0.449947,call
1,0.180795,reach
2,0.0813629,move
3,0.0631731,thanks
4,0.0479217,greet
5,0.0419101,attention
6,0.0364903,goodbye
7,0.0313377,status
8,0.0309243,joke
9,0.0231384,deny


In [25]:
#Script to get the Text-To-Speech output.
audioFolder = "./audios/"
effect = ["",
          "&FX_TYPE=P&FX_LEVEL=2", #Pitch
          "&FX_TYPE=D&FX_LEVEL=-1" #Duration
         ]
rooturl = ["https://cache-a.oddcast.com/tts/gen.php?EID=3&LID=1&VID=3&TXT=",
           "&IS_UTF8=1&ACC=3314795&API=2292376&CB=vw_mc.vwCallback&HTTP_ERR=1&vwApiVersion=2"]
characters ={"," : "%2C"," " : "%20","?" : "%3F","'" : "%27"}
def play(phrase, ef = 0):
    for ch in characters:
        phrase = phrase.replace(ch,characters[ch]) #Replace character to match type in URL
    url = rooturl[0] + phrase + effect[ef]+ rooturl[1] #Build the URL
    file = audioFolder + str(uuid.uuid4()) +'.mp3'
    urllib.request.urlretrieve(url, file)
    playsound(file, True)
    os.remove(file)

In [26]:
play("My name is nurse. How may I help you?",2)

In [17]:
#Configure microphone calibration
# Perform a quick test
r = sr.Recognizer()
with sr.Microphone() as source:
    #print("Please wait. Calibrating microphone...")
    # listen for 1 second and create the ambient noise energy level
    r.adjust_for_ambient_noise(source, duration=1)
    print("Say something!")
    audio = r.listen(source,phrase_time_limit=5)
    response = r.recognize_google(audio)
    print("I think you said: " + response)

Say something!


UnknownValueError: 

In [27]:
def printOut(msg, speech_out = False):
    print("👩 "+" >> " + msg +"\n")
    if(speech_out):
        play(msg)

In [28]:
def init_nurse_bot(speech_in=False,speech_out=False):
    print("[INFO] The nurse is ready to listen. Please start a dialog... (Type 'stop' to quit)\n")
    errormsg = "I'm sorry, I didn't get it. Could you repeat please?"
    with sr.Microphone() as source:
        r.adjust_for_ambient_noise(source, duration=1)
        while True:
            # >>>>>>>>>>> Input sentence
            if(speech_in):
                flag = False
                cont = 0
                while(not flag):
                    try:
                        print("Listening... >> ", end='')
                        audio = r.listen(source,phrase_time_limit=3)
                        a = r.recognize_google(audio)
                        print(a)
                        flag = True
                    except:
                        cont += 1
                        printOut(errormsg, False)
                        if(cont == 3):
                            print("\n[TERMINATED]", end='')
                            return
            else:
                print(">> ", end='')
                a = input()
            if a == 'stop':
                print("\n[TERMINATED]", end='')
                return
            responses = agent.handle_message(a)
            if(len(responses) == 0):
                print("Action returned 0 elements. Maybe the action listener is not active. Activate listener:\n python -m rasa_core_sdk.endpoint --actions actions")
            for response in responses:
                printOut(response["text"], speech_out)
                if(response["text"] == "Have a nice day!"):
                    print("\n[TERMINATED]", end='')
                    return

In [36]:
init_nurse_bot(False,False)

[INFO] The nurse is ready to listen. Please start a dialog... (Type 'stop' to quit)

>> Hello
👩  >> Hi human. You can call me Nurse. How may I help you?

>> I feel fine
👩  >> Anytime!

👩  >> Is there something else I can do for you?

>> i am feeling bad
Action returned 0 elements. Maybe the action listener is not active. Activate listener:
 python -m rasa_core_sdk.endpoint --actions actions
>> I feel pain
👩  >> That's not good. Let me get some medicine, and I will call a human to verify your condition.

>> stop

[TERMINATED]

In [29]:
#!python -m rasa_core.run -d models/dialogue -u models/nursebot/interpreter --debug --endpoints endpoints.yml